# EDA on Light Training, Time of Year(Covid Shutdown)

In [357]:
#Light Training began on 2/15/18
#3/14/20 - 5/9/20 Closed to Public

In [358]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import datetime
import plotly.graph_objects as go
import scipy.stats as st

In [359]:
eat = pd.read_excel("SharkData.xlsx", sheet_name = 0)
targets = pd.read_excel("SharkData.xlsx", sheet_name = 1)
drops = pd.read_excel("SharkData.xlsx", sheet_name = 2)
other_factors = pd.read_excel("SharkData.xlsx", sheet_name = 3)

Cleaning for each each Dataframe

In [360]:
eat = eat.iloc[1:,:-1]
targets = targets.iloc[1:,:-1]
drops = drops.iloc[1:,:-1]
eat.rename(columns = {"Unnamed: 14":'Etc. Comments', "Pieces Eaten": "Date"}, inplace = True)
targets.rename(columns = {"Targets": "Date"}, inplace = True)
drops.rename(columns = {"Drops": "Date"}, inplace = True)

#get rid of null values at end of dataset
eat = eat.head(582)
targets = targets.head(582)
drops = drops.head(582)

# Change date to just date format, not datetime
eat['Date'] = pd.to_datetime(eat['Date']).dt.date
targets['Date'] = pd.to_datetime(targets['Date']).dt.date
drops['Date'] = pd.to_datetime(drops['Date']).dt.date


eat.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,Etc. Comments
1,2017-12-05,0.0,5.0,3.0,6.0,5.0,4.0,4.0,0.0,0.0,2.0,3.0,1.0,33.0,NaN
2,2017-12-07,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,NaN
3,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,3.0,1.0,0.0,6.0,NaN
4,2017-12-10,5.0,1.0,1.0,3.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,14.0,NaN
5,2017-12-12,2.0,2.0,0.0,5.0,2.0,4.0,5.0,0.0,2.0,3.0,2.0,0.0,27.0,NaN


In [361]:
targets.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:
1,2017-12-05,3.0,0.0,4.0,3.0,0.0,2.0,2.0,1.0,1.0,3.0,3.0,0.0,22.0
2,2017-12-07,4.0,5.0,2.0,2.0,5.0,3.0,5.0,0.0,0.0,2.0,2.0,0.0,30.0
3,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,1.0,0.0,3.0,8.0
4,2017-12-10,4.0,7.0,0.0,2.0,2.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,15.0
5,2017-12-12,3.0,1.0,3.0,2.0,4.0,1.0,1.0,1.0,1.0,5.0,1.0,3.0,26.0


In [362]:
drops.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:
1,2017-12-05,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,3.0
2,2017-12-07,0.0,1.0,1.0,2.0,1.0,1.0,2.0,0.0,0.0,2.0,0.0,0.0,10.0
3,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,1.0
4,2017-12-10,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0
5,2017-12-12,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0


Add the temperature to each Dataframe

In [363]:
other_factors['Temperature'].count()

582

In [364]:
#eat['Temperature'] = other_factors['Temperature'].values
#targets['Temperature'] = other_factors['Temperature'].values
#drops['Temperature'] = other_factors['Temperature'].values

In [365]:
eat

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,Etc. Comments
1,2017-12-05,0.0,5.0,3.0,6.0,5.0,4.0,4.0,0.0,0.0,2.0,3.0,1.0,33.0,NaN
2,2017-12-07,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,NaN
3,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,3.0,1.0,0.0,6.0,NaN
4,2017-12-10,5.0,1.0,1.0,3.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,14.0,NaN
5,2017-12-12,2.0,2.0,0.0,5.0,2.0,4.0,5.0,0.0,2.0,3.0,2.0,0.0,27.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,2020-09-22,6.0,0.0,1.0,2.0,6.0,0.0,4.0,1.0,0.0,0.0,5.0,1.0,26.0,NaN
579,2020-09-24,0.0,9.0,0.0,7.0,5.0,5.0,7.0,0.0,1.0,2.0,2.0,6.0,44.0,NaN
580,2020-09-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,0.0,2.0,3.0,3.0,12.0,NaN
581,2020-09-27,5.0,6.0,3.0,6.0,4.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,33.0,NaN


Divide data according to Light Training vs Non-Light Training
-Light training starts on Row 42

In [366]:
eat_no_training = eat.head(41)
eat_training = eat[42:582]
targets_no_training = targets.head(41)
targets_training = targets[42:582]
drops_no_training = drops.head(41)
drops_training = drops[42:582]

In [367]:
drops_training

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:
43,2018-02-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
44,2018-02-18,1.0,2.0,1.0,0.0,2.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,7.0
45,2018-02-20,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,7.0
46,2018-02-22,0.0,0.0,1.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,6.0
47,2018-02-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,2020-09-22,0.0,0.0,1.0,4.0,0.0,2.0,2.0,0.0,0.0,0.0,1.0,0.0,10.0
579,2020-09-24,0.0,0.0,0.0,3.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,7.0
580,2020-09-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,1.0
581,2020-09-27,0.0,0.0,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,3.0


I am going to seperate the data into three different groups based on Pre-covid(1-470), covid(471-499), post_covid(500-582)

In [368]:
eat_pre = eat[0:470]
eat_covid = eat[471:499]
eat_post = eat[500:582]
targets_pre = targets[1:470]
targets_covid = targets[471:499]
targets_post = targets[500:582]
drops_pre = drops[1:470]
drops_covid = drops[471:499]
drops_post = drops[500:582]

# Eat EDA

In [369]:
all_sharks_pre = eat_pre[['Ross', 'Chandler','BT1','BT2','BT3','BT4','BT5','GR1','GR2','GR3','GR4', 'GR5']]
all_sharks_covid = eat_covid[['Ross', 'Chandler','BT1','BT2','BT3','BT4','BT5','GR1','GR2','GR3','GR4', 'GR5']]
all_sharks_post = eat_post[['Ross', 'Chandler','BT1','BT2','BT3','BT4','BT5','GR1','GR2','GR3','GR4', 'GR5']]

In [370]:
all_mean_pre = round(all_sharks_pre.mean(), 3)
all_mean_covid = round(all_sharks_covid.mean(), 3)
all_mean_post = round(all_sharks_post.mean(), 3)
all_mean_pre

Ross        3.456
Chandler    2.811
BT1         2.088
BT2         2.691
BT3         2.323
BT4         2.703
BT5         3.040
GR1         1.266
GR2         1.391
GR3         1.909
GR4         1.635
GR5         1.683
dtype: float64

In [371]:
all_mean_covid

Ross        2.609
Chandler    2.652
BT1         1.826
BT2         4.304
BT3         1.435
BT4         2.696
BT5         2.565
GR1         1.048
GR2         1.000
GR3         1.762
GR4         1.286
GR5         1.143
dtype: float64

In [372]:
all_mean_post

Ross        4.452
Chandler    3.371
BT1         1.871
BT2         3.274
BT3         2.839
BT4         3.323
BT5         3.468
GR1         1.869
GR2         1.279
GR3         1.574
GR4         1.525
GR5         2.230
dtype: float64

In [373]:
# Graph Mean Number of Fish Eaten For Each Shark for each time pre, during, post covid.

eaten_means = {'Shark': all_sharks_pre.columns,
               'Pre_Mean': all_mean_pre,
               'Covid_Mean': all_mean_covid,
               'Post_Mean': all_mean_post}

eaten_means_df = pd.DataFrame(eaten_means, columns = ['Shark', 'Pre_Mean', 'Covid_Mean','Post_Mean'])

In [374]:
pre_eaten_means_sorted = eaten_means_df.sort_values(by=['Pre_Mean'], ascending = False)
covid_eaten_means_sorted = eaten_means_df.sort_values(by=['Covid_Mean'], ascending = False)
post_eaten_means_sorted = eaten_means_df.sort_values(by=['Post_Mean'], ascending = False)

In [375]:
fig = px.histogram(pre_eaten_means_sorted, x="Shark", y="Pre_Mean", 
                   title = "Mean Number of Fish Eaten by Shark before Shutdown",
                  labels={
                     "Shark": "Shark",
                     "Mean": " Mean Number of Fish",
                 })
fig.show()

In [376]:

fig = px.histogram(covid_eaten_means_sorted, x="Shark", y="Covid_Mean", 
                   title = "Mean Number of Fish Eaten by Shark during Shutdown",
                  labels={
                     "Shark": "Shark",
                     "Mean": " Mean Number of Fish",
                 })
fig.show()

In [377]:

fig = px.histogram(post_eaten_means_sorted, x="Shark", y="Post_Mean", 
                   title = "Mean Number of Fish Eaten by Shark after Reopening",
                  labels={
                     "Shark": "Shark",
                     "Mean": " Mean Number of Fish",
                 })
fig.show()

In [378]:
# Shark feeding vector pre covid
total_sharks_feeding_pre = eat_pre[["Ross","Chandler","BT1","BT2","BT3","BT4","BT5","GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
BT_sharks_feeding_pre = eat_pre[["BT1","BT2","BT3","BT4","BT5"]].count(axis = 1)
GR_sharks_feeding_pre = eat_pre[["GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
SS_sharks_feeding_pre = eat_pre[["Ross","Chandler"]].count(axis = 1)
male_sharks_feeding_pre = eat_pre[["Ross","Chandler","BT1","BT5","GR1"]].count(axis = 1)
female_sharks_feeding_pre = eat_pre[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].count(axis = 1)

# Shark feeding vector during covid
total_sharks_feeding_covid = eat_covid[["Ross","Chandler","BT1","BT2","BT3","BT4","BT5","GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
BT_sharks_feeding_covid = eat_covid[["BT1","BT2","BT3","BT4","BT5"]].count(axis = 1)
GR_sharks_feeding_covid = eat_covid[["GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
SS_sharks_feeding_covid = eat_covid[["Ross","Chandler"]].count(axis = 1)
male_sharks_feeding_covid = eat_covid[["Ross","Chandler","BT1","BT5","GR1"]].count(axis = 1)
female_sharks_feeding_covid = eat_covid[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].count(axis = 1)

# Shark feeding vector post covid
total_sharks_feeding_post = eat_post[["Ross","Chandler","BT1","BT2","BT3","BT4","BT5","GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
BT_sharks_feeding_post = eat_post[["BT1","BT2","BT3","BT4","BT5"]].count(axis = 1)
GR_sharks_feeding_post = eat_post[["GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
SS_sharks_feeding_post = eat_post[["Ross","Chandler"]].count(axis = 1)
male_sharks_feeding_post = eat_post[["Ross","Chandler","BT1","BT5","GR1"]].count(axis = 1)
female_sharks_feeding_post = eat_post[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].count(axis = 1)
total_sharks_feeding_post

501     7
502    12
503    12
504     5
505     7
       ..
578    12
579    12
580     5
581     7
582    12
Length: 82, dtype: int64

In [379]:
eat_pre["male"] = eat_pre[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
eat_pre["female"] = eat_pre[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)
eat_covid["male"] = eat_covid[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
eat_covid["female"] = eat_covid[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)
eat_post["male"] = eat_post[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
eat_post["female"] = eat_post[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)

eat_pre["All_GR"] = eat_pre["GR1"] + eat_pre["GR2"] + eat_pre["GR3"]+ eat_pre["GR4"] + eat_pre["GR5"]
eat_pre["All_BT"] = eat_pre["BT1"] + eat_pre["BT2"] + eat_pre["BT3"]+ eat_pre["BT4"] + eat_pre["BT5"]
eat_pre["All_SS"] = eat_pre["Ross"] + eat_pre["Chandler"]

eat_covid["All_GR"] = eat_covid["GR1"] + eat_covid["GR2"] + eat_covid["GR3"]+ eat_covid["GR4"] + eat_covid["GR5"]
eat_covid["All_BT"] = eat_covid["BT1"] + eat_covid["BT2"] + eat_covid["BT3"]+ eat_covid["BT4"] + eat_covid["BT5"]
eat_covid["All_SS"] = eat_covid["Ross"] + eat_covid["Chandler"]

eat_post["All_GR"] = eat_post["GR1"] + eat_post["GR2"] + eat_post["GR3"]+ eat_post["GR4"] + eat_post["GR5"]
eat_post["All_BT"] = eat_post["BT1"] + eat_post["BT2"] + eat_post["BT3"]+ eat_post["BT4"] + eat_post["BT5"]
eat_post["All_SS"] = eat_post["Ross"] + eat_post["Chandler"]

<ipython-input-379-6d17c63af8cb>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-379-6d17c63af8cb>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-379-6d17c63af8cb>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-379-6d1

In [380]:
#Average Mean Graphs Gender - Pre

all_sharks_eat_pre = eat_pre[["female","male"]]
all_sharks_eat_pre["female"] = all_sharks_eat_pre["female"] / female_sharks_feeding_pre
all_sharks_eat_pre["male"] = all_sharks_eat_pre["male"] / male_sharks_feeding_pre
eat_pre_means = pd.DataFrame(round(all_sharks_eat_pre.mean(), 3)).reset_index()
eat_pre_means.columns = ["Gender","Mean"]
eat_pre_means = eat_pre_means.sort_values(by = "Mean", ascending = False)
print(eat_pre_means) 
px.bar(eat_pre_means, x = "Gender", y = "Mean", title = "How often is the average shark eating for each gender before Covid?")

   Gender   Mean
1    male  2.385
0  female  2.120


<ipython-input-380-ec069a954aef>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-380-ec069a954aef>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [381]:
#Average Mean Graphs Gender - During Covid

all_sharks_eat_covid = eat_covid[["female","male"]]
all_sharks_eat_covid["female"] = all_sharks_eat_covid["female"] / female_sharks_feeding_covid
all_sharks_eat_covid["male"] = all_sharks_eat_covid["male"] / male_sharks_feeding_covid
eat_covid_means = pd.DataFrame(round(all_sharks_eat_covid.mean(), 3)).reset_index()
eat_covid_means.columns = ["Gender","Mean"]
eat_covid_means = eat_covid_means.sort_values(by = "Mean", ascending = False)
print(eat_covid_means) 
px.bar(eat_covid_means, x = "Gender", y = "Mean", title = "How often is the average shark eating for each gender without patrons?")

   Gender   Mean
0  female  2.170
1    male  2.043


<ipython-input-381-7465f17d6367>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-381-7465f17d6367>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [382]:
#Average Mean Graphs Gender - Post

all_sharks_eat_post = eat_post[["female","male"]]
all_sharks_eat_post["female"] = all_sharks_eat_post["female"] / female_sharks_feeding_post
all_sharks_eat_post["male"] = all_sharks_eat_post["male"] / male_sharks_feeding_post
eat_post_means = pd.DataFrame(round(all_sharks_eat_post.mean(), 3)).reset_index()
eat_post_means.columns = ["Gender","Mean"]
eat_post_means = eat_post_means.sort_values(by = "Mean", ascending = False)
print(eat_post_means) 
px.bar(eat_post_means, x = "Gender", y = "Mean", title = "How often is the average shark eating for each gender after shutdwon")

<ipython-input-382-41e243802010>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-382-41e243802010>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



   Gender   Mean
1    male  2.770
0  female  2.259


In [383]:
eat_pre = eat_pre.drop('Etc. Comments', axis=1)
eat_covid = eat_covid.drop('Etc. Comments', axis=1)
eat_post = eat_post.drop('Etc. Comments', axis=1)
eat_pre

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,male,female,All_GR,All_BT,All_SS
1,2017-12-05,0.0,5.0,3.0,6.0,5.0,4.0,4.0,0.0,0.0,2.0,3.0,1.0,33.0,12.0,21.0,6.0,22.0,5.0
2,2017-12-07,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,2.0,0.0,2.0,1.0
3,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,3.0,1.0,0.0,6.0,2.0,4.0,6.0,NaN,NaN
4,2017-12-10,5.0,1.0,1.0,3.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,14.0,7.0,7.0,NaN,8.0,6.0
5,2017-12-12,2.0,2.0,0.0,5.0,2.0,4.0,5.0,0.0,2.0,3.0,2.0,0.0,27.0,9.0,18.0,7.0,16.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,2020-03-05,8.0,0.0,0.0,1.0,0.0,1.0,5.0,0.0,0.0,0.0,1.0,0.0,16.0,13.0,3.0,1.0,7.0,8.0
467,2020-03-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,4.0,2.0,4.0,11.0,0.0,11.0,11.0,NaN,NaN
468,2020-03-08,0.0,8.0,9.0,0.0,2.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,21.0,19.0,2.0,NaN,13.0,8.0
469,2020-03-10,1.0,3.0,0.0,4.0,3.0,2.0,6.0,3.0,0.0,4.0,1.0,2.0,29.0,13.0,16.0,10.0,15.0,4.0


In [384]:
#Average Mean Graphs Per Species - Pre

all_sharks_eat_pre = eat_pre[["All_GR","All_BT","All_SS"]]
all_sharks_eat_pre["All_GR"] = all_sharks_eat_pre["All_GR"] / GR_sharks_feeding_pre
all_sharks_eat_pre["All_BT"] = all_sharks_eat_pre["All_BT"] / BT_sharks_feeding_pre
all_sharks_eat_pre["All_SS"] = all_sharks_eat_pre["All_SS"] / SS_sharks_feeding_pre
all_sharks_eat_pre.columns = ["Grey Reef","Black Tip","Sand Shark"]
eat_pre_means = pd.DataFrame(round(all_sharks_eat_pre.mean(), 3)).reset_index()
eat_pre_means.columns = ["Species","Mean"]
eat_pre_means = eat_pre_means.sort_values(by = "Mean", ascending = False)
print(eat_pre_means)
px.bar(eat_pre_means, x = "Species", y = "Mean", title = "Eat Means for Each species on average before covid")

      Species   Mean
2  Sand Shark  3.137
1   Black Tip  2.572
0   Grey Reef  1.577


<ipython-input-384-98dd1407732c>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-384-98dd1407732c>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-384-98dd1407732c>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [385]:
#Average Mean Graphs Per Species - During

all_sharks_eat_covid = eat_covid[["All_GR","All_BT","All_SS"]]
all_sharks_eat_covid["All_GR"] = all_sharks_eat_covid["All_GR"] / GR_sharks_feeding_covid
all_sharks_eat_covid["All_BT"] = all_sharks_eat_covid["All_BT"] / BT_sharks_feeding_covid
all_sharks_eat_covid["All_SS"] = all_sharks_eat_covid["All_SS"] / SS_sharks_feeding_covid
all_sharks_eat_covid.columns = ["Grey Reef","Black Tip","Sand Shark"]
eat_covid_means = pd.DataFrame(round(all_sharks_eat_covid.mean(), 3)).reset_index()
eat_covid_means.columns = ["Species","Mean"]
eat_covid_means = eat_covid_means.sort_values(by = "Mean", ascending = False)
print(eat_covid_means)
px.bar(eat_covid_means, x = "Species", y = "Mean", title = "Eat Means for Each species on average during shutdown")

<ipython-input-385-e4a025c86205>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-385-e4a025c86205>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-385-e4a025c86205>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



      Species   Mean
2  Sand Shark  2.630
1   Black Tip  2.565
0   Grey Reef  1.248


In [386]:
#Average Mean Graphs Per Species post - Post

all_sharks_eat_post = eat_post[["All_GR","All_BT","All_SS"]]
all_sharks_eat_post["All_GR"] = all_sharks_eat_post["All_GR"] / GR_sharks_feeding_post
all_sharks_eat_post["All_BT"] = all_sharks_eat_post["All_BT"] / BT_sharks_feeding_post
all_sharks_eat_post["All_SS"] = all_sharks_eat_post["All_SS"] / SS_sharks_feeding_post
all_sharks_eat_post.columns = ["Grey Reef","Black Tip","Sand Shark"]
eat_post_means = pd.DataFrame(round(all_sharks_eat_post.mean(), 3)).reset_index()
eat_post_means.columns = ["Species","Mean"]
eat_post_means = eat_post_means.sort_values(by = "Mean", ascending = False)
print(eat_post_means)
px.bar(eat_post_means, x = "Species", y = "Mean", title = "Eat Means for Each species on average after reopening")

      Species   Mean
2  Sand Shark  3.911
1   Black Tip  2.955
0   Grey Reef  1.695


<ipython-input-386-719e0ff8b8e7>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-386-719e0ff8b8e7>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-386-719e0ff8b8e7>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [387]:
All_Sharks_eat_pre = eat_pre.iloc[:,1:]
All_Sharks_eat_pre["All_GR"] = All_Sharks_eat_pre["All_GR"] / GR_sharks_feeding_pre
All_Sharks_eat_pre["All_BT"] = All_Sharks_eat_pre["All_BT"] / BT_sharks_feeding_pre
All_Sharks_eat_pre["All_SS"] = All_Sharks_eat_pre["All_SS"] / SS_sharks_feeding_pre
All_Sharks_eat_pre["female"] = All_Sharks_eat_pre["female"] / female_sharks_feeding_pre
All_Sharks_eat_pre["male"] = All_Sharks_eat_pre["male"] / male_sharks_feeding_pre
All_Sharks_eat_pre["Total: "] = All_Sharks_eat_pre["Total: "] / total_sharks_feeding_pre
eat_pre_means = pd.DataFrame(round(All_Sharks_eat_pre.mean(), 3)).reset_index()
eat_pre_means.columns = ["Subset","Mean"]
eat_pre_means = eat_pre_means.sort_values(by = "Mean", ascending = False)
ci = {}
for column in eat_pre.iloc[:,1:].columns:
    conf = st.t.interval(alpha=0.95, df=len(eat_pre[column])-1, loc=np.mean(eat_pre[column]), scale=st.sem(eat_pre[column],nan_policy='omit'))
    error = (conf[1]-conf[0])/2
    ci.update({column:error})
ci = pd.DataFrame(list(ci.items()),columns = ['Subset','SE'])
eat_pre_means = pd.merge(eat_pre_means, ci, how = 'outer', on = "Subset")
#eat_pre_means = pd.merge(eat_pre_means,pd.melt(eat_pre.describe().loc[["std"]], var_name = "Subset",value_name = "Standard Deviation"), how = "outer", on = "Subset")
prop_average = eat_pre_means[eat_pre_means["Subset"]=="Total: "]["Mean"].values[0]
Average = []
for i in eat_pre_means.Mean:
    if i > prop_average:
        Average.append("Above Average")
    elif i == prop_average:
        Average.append("Average")
    else:
        Average.append("Below Average")
eat_pre_means["Average"] = Average
max_y = (eat_pre_means["Mean"] + eat_pre_means["SE"]).max() 
#max_y = (eat_pre_means["Mean"] + eat_pre_means["Standard Deviation"]).max() 
px.bar(eat_pre_means, x = "Subset", y = "Mean", color = "Average",text = "Mean", error_y = "SE",title = "Eat Means for each subset with Confidence Intervals before covid").update_traces(textposition='outside')

In [464]:
eat_pre

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,male,female,All_GR,All_BT,All_SS
1,2017-12-05,0.0,5.0,3.0,6.0,5.0,4.0,4.0,0.0,0.0,2.0,3.0,1.0,33.0,12.0,21.0,6.0,22.0,5.0
2,2017-12-07,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,2.0,0.0,2.0,1.0
3,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,3.0,1.0,0.0,6.0,2.0,4.0,6.0,NaN,NaN
4,2017-12-10,5.0,1.0,1.0,3.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,14.0,7.0,7.0,NaN,8.0,6.0
5,2017-12-12,2.0,2.0,0.0,5.0,2.0,4.0,5.0,0.0,2.0,3.0,2.0,0.0,27.0,9.0,18.0,7.0,16.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,2020-03-05,8.0,0.0,0.0,1.0,0.0,1.0,5.0,0.0,0.0,0.0,1.0,0.0,16.0,13.0,3.0,1.0,7.0,8.0
467,2020-03-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,4.0,2.0,4.0,11.0,0.0,11.0,11.0,NaN,NaN
468,2020-03-08,0.0,8.0,9.0,0.0,2.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,21.0,19.0,2.0,NaN,13.0,8.0
469,2020-03-10,1.0,3.0,0.0,4.0,3.0,2.0,6.0,3.0,0.0,4.0,1.0,2.0,29.0,13.0,16.0,10.0,15.0,4.0


In [465]:
eat_pre.iloc[:,1:]

,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,male,female,All_GR,All_BT,All_SS
1,0.0,5.0,3.0,6.0,5.0,4.0,4.0,0.0,0.0,2.0,3.0,1.0,33.0,12.0,21.0,6.0,22.0,5.0
2,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,2.0,0.0,2.0,1.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,3.0,1.0,0.0,6.0,2.0,4.0,6.0,NaN,NaN
4,5.0,1.0,1.0,3.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,14.0,7.0,7.0,NaN,8.0,6.0
5,2.0,2.0,0.0,5.0,2.0,4.0,5.0,0.0,2.0,3.0,2.0,0.0,27.0,9.0,18.0,7.0,16.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,8.0,0.0,0.0,1.0,0.0,1.0,5.0,0.0,0.0,0.0,1.0,0.0,16.0,13.0,3.0,1.0,7.0,8.0
467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,4.0,2.0,4.0,11.0,0.0,11.0,11.0,NaN,NaN
468,0.0,8.0,9.0,0.0,2.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,21.0,19.0,2.0,NaN,13.0,8.0
469,1.0,3.0,0.0,4.0,3.0,2.0,6.0,3.0,0.0,4.0,1.0,2.0,29.0,13.0,16.0,10.0,15.0,4.0


In [466]:
eat_covid

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,male,female,All_GR,All_BT,All_SS
472,2020-03-15,2.0,10.0,3.0,1.0,0.0,2.0,8.0,NaN,NaN,NaN,NaN,NaN,26.0,23.0,3.0,NaN,14.0,12.0
473,2020-03-17,5.0,0.0,0.0,7.0,2.0,2.0,6.0,3.0,1.0,0.0,0.0,0.0,26.0,14.0,12.0,4.0,17.0,5.0
474,2020-03-19,5.0,1.0,3.0,1.0,1.0,1.0,3.0,2.0,0.0,4.0,0.0,1.0,22.0,14.0,8.0,7.0,9.0,6.0
475,2020-03-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0,0.0,2.0,2.0,6.0,0.0,6.0,6.0,NaN,NaN
476,2020-03-22,1.0,6.0,1.0,8.0,2.0,8.0,2.0,NaN,NaN,NaN,NaN,NaN,28.0,10.0,18.0,NaN,21.0,7.0
477,2020-03-26,3.0,3.0,2.0,8.0,2.0,14.0,3.0,1.0,1.0,2.0,1.0,1.0,41.0,12.0,29.0,6.0,29.0,6.0
478,2020-03-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.0,4.0,0.0,5.0,16.0,4.0,12.0,16.0,NaN,NaN
479,2020-03-29,0.0,2.0,3.0,1.0,3.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,12.0,6.0,6.0,NaN,10.0,2.0
480,2020-03-31,0.0,2.0,2.0,2.0,3.0,1.0,1.0,0.0,2.0,0.0,0.0,1.0,14.0,5.0,9.0,3.0,9.0,2.0
481,2020-04-02,1.0,4.0,1.0,4.0,1.0,3.0,3.0,2.0,1.0,2.0,2.0,2.0,26.0,11.0,15.0,9.0,12.0,5.0


In [467]:
eat_covid.iloc[:,1:]

,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,male,female,All_GR,All_BT,All_SS
472,2.0,10.0,3.0,1.0,0.0,2.0,8.0,NaN,NaN,NaN,NaN,NaN,26.0,23.0,3.0,NaN,14.0,12.0
473,5.0,0.0,0.0,7.0,2.0,2.0,6.0,3.0,1.0,0.0,0.0,0.0,26.0,14.0,12.0,4.0,17.0,5.0
474,5.0,1.0,3.0,1.0,1.0,1.0,3.0,2.0,0.0,4.0,0.0,1.0,22.0,14.0,8.0,7.0,9.0,6.0
475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0,0.0,2.0,2.0,6.0,0.0,6.0,6.0,NaN,NaN
476,1.0,6.0,1.0,8.0,2.0,8.0,2.0,NaN,NaN,NaN,NaN,NaN,28.0,10.0,18.0,NaN,21.0,7.0
477,3.0,3.0,2.0,8.0,2.0,14.0,3.0,1.0,1.0,2.0,1.0,1.0,41.0,12.0,29.0,6.0,29.0,6.0
478,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.0,4.0,0.0,5.0,16.0,4.0,12.0,16.0,NaN,NaN
479,0.0,2.0,3.0,1.0,3.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,12.0,6.0,6.0,NaN,10.0,2.0
480,0.0,2.0,2.0,2.0,3.0,1.0,1.0,0.0,2.0,0.0,0.0,1.0,14.0,5.0,9.0,3.0,9.0,2.0
481,1.0,4.0,1.0,4.0,1.0,3.0,3.0,2.0,1.0,2.0,2.0,2.0,26.0,11.0,15.0,9.0,12.0,5.0


In [469]:
all_sharks_eat_covid = eat_covid.iloc[:,1:]
all_sharks_eat_covid["All_GR"] = all_sharks_eat_covid["All_GR"] / GR_sharks_feeding_covid
all_sharks_eat_covid["All_BT"] = all_sharks_eat_covid["All_BT"] / BT_sharks_feeding_covid
all_sharks_eat_covid["All_SS"] = all_sharks_eat_covid["All_SS"] / SS_sharks_feeding_covid
all_sharks_eat_covid["female"] = all_sharks_eat_covid["female"] / female_sharks_feeding_covid
all_sharks_eat_covid["male"] = all_sharks_eat_covid["male"] / male_sharks_feeding_covid
all_sharks_eat_covid["Total: "] = all_sharks_eat_covid["Total: "] / total_sharks_feeding_covid
eat_covid_means = pd.DataFrame(round(all_sharks_eat_covid.mean(), 3)).reset_index()
eat_covid_means.columns = ["Subset","Mean"]
eat_covid_means = eat_covid_means.sort_values(by = "Mean", ascending = False)
ci = {}
for column in eat_covid.iloc[:,1:].columns:
    conf = st.t.interval(alpha=0.95, df=len(eat_covid[column])-1, loc=np.mean(eat_covid[column]), scale=st.sem(eat_covid[column],nan_policy='omit'))
    error = (conf[1]-conf[0])/2
    ci.update({column:error})
ci = pd.DataFrame(list(ci.items()),columns = ['Subset','SE'])
eat_covid_means = pd.merge(eat_covid_means, ci, how = 'outer', on = "Subset")
#eat_covid_means = pd.merge(eat_covid_means,pd.melt(eat_covid.describe().loc[["std"]], var_name = "Subset",value_name = "Standard Deviation"), how = "outer", on = "Subset")
prop_average = eat_covid_means[eat_covid_means["Subset"]=="Total: "]["Mean"].values[0]
Average = []
for i in eat_covid_means.Mean:
    if i > prop_average:
        Average.append("Above Average")
    elif i == prop_average:
        Average.append("Average")
    else:
        Average.append("Below Average")
eat_covid_means["Average"] = Average
max_y = (eat_covid_means["Mean"] + eat_covid_means["SE"]).max() 
#max_y = (eat_covid_means["Mean"] + eat_covid_means["Standard Deviation"]).max() 
px.bar(eat_covid_means, x = "Subset", y = "Mean", color = "Average",text = "Mean", error_y = "SE",title = "Eat Means for each subset with Confidence Intervals during shutdown").update_traces(textposition='outside')

In [389]:
All_Sharks_eat_post = eat_post.iloc[:,1:]
All_Sharks_eat_post["All_GR"] = All_Sharks_eat_post["All_GR"] / GR_sharks_feeding_post
All_Sharks_eat_post["All_BT"] = All_Sharks_eat_post["All_BT"] / BT_sharks_feeding_post
All_Sharks_eat_post["All_SS"] = All_Sharks_eat_post["All_SS"] / SS_sharks_feeding_post
All_Sharks_eat_post["female"] = All_Sharks_eat_post["female"] / female_sharks_feeding_post
All_Sharks_eat_post["male"] = All_Sharks_eat_post["male"] / male_sharks_feeding_post
All_Sharks_eat_post["Total: "] = All_Sharks_eat_post["Total: "] / total_sharks_feeding_post
eat_post_means = pd.DataFrame(round(All_Sharks_eat_post.mean(), 3)).reset_index()
eat_post_means.columns = ["Subset","Mean"]
eat_post_means = eat_post_means.sort_values(by = "Mean", ascending = False)
ci = {}
for column in eat_post.iloc[:,1:].columns:
    conf = st.t.interval(alpha=0.95, df=len(eat_post[column])-1, loc=np.mean(eat_post[column]), scale=st.sem(eat_post[column],nan_policy='omit'))
    error = (conf[1]-conf[0])/2
    ci.update({column:error})
ci = pd.DataFrame(list(ci.items()),columns = ['Subset','SE'])
eat_post_means = pd.merge(eat_post_means, ci, how = 'outer', on = "Subset")
#eat_post_means = pd.merge(eat_post_means,pd.melt(eat_post.describe().loc[["std"]], var_name = "Subset",value_name = "Standard Deviation"), how = "outer", on = "Subset")
prop_average = eat_post_means[eat_post_means["Subset"]=="Total: "]["Mean"].values[0]
Average = []
for i in eat_post_means.Mean:
    if i > prop_average:
        Average.append("Above Average")
    elif i == prop_average:
        Average.append("Average")
    else:
        Average.append("Below Average")
eat_post_means["Average"] = Average
max_y = (eat_post_means["Mean"] + eat_post_means["SE"]).max() 
#max_y = (eat_post_means["Mean"] + eat_post_means["Standard Deviation"]).max() 
px.bar(eat_post_means, x = "Subset", y = "Mean", color = "Average",text = "Mean", error_y = "SE",title = "Eat Means for each subset with Confidence Intervals after Shutdown").update_traces(textposition='outside')

In [390]:
eat_pre_prop = eat_pre.iloc[:,1:]
eat_covid_prop = eat_covid.iloc[:,1:]
eat_post_prop = eat_post.iloc[:,1:]

In [391]:
props_pre = {}
for i in eat_pre_prop.columns:
    data = eat_pre_prop[i]
    eat_pre_days = data > 0
    props_pre.update({i:(round(len(data[eat_pre_days])/len(data[data.isna()==False]),2))})
props_pre = pd.DataFrame(list(props_pre.items()),columns = ['Subset of Data','Proportion of Days Eaten before Covid'])

In [392]:
props_covid = {}
for i in eat_pre_prop.columns:
    data = eat_pre_prop[i]
    eat_pre_days = data > 0
    props_covid.update({i:(round(len(data[eat_pre_days])/len(data[data.isna()==False]),2))})
props_covid = pd.DataFrame(list(props_covid.items()),columns = ['Subset of Data','Proportion of Days Eaten during Shutdown'])

In [393]:
props_post = {}
for i in eat_pre_prop.columns:
    data = eat_pre_prop[i]
    eat_pre_days = data > 0
    props_post.update({i:(round(len(data[eat_pre_days])/len(data[data.isna()==False]),2))})
props_post = pd.DataFrame(list(props_post.items()),columns = ['Subset of Data','Proportion of Days Eaten after Reopening'])

In [394]:
print(props_pre.iloc[12].values[1],"is the overall proportion of days where a shark ate before covid")
print(props_covid.iloc[12].values[1],"is the overall proportion of days where a shark ate during shutdown")
print(props_post.iloc[12].values[1],"is the overall proportion of days where a shark ate after aquarium reopened")

0.99 is the overall proportion of days where a shark ate before covid
0.99 is the overall proportion of days where a shark ate during shutdown
0.99 is the overall proportion of days where a shark ate after aquarium reopened


In [395]:
# Proportion before covid
eat_pre_prop_species = props_pre.iloc[15:,:].rename(columns = {"Subset of Data":"Species"}).sort_values(by = "Proportion of Days Eaten before Covid", ascending = False)
eat_pre_prop_gender = props_pre.iloc[13:15,:].rename(columns = {"Subset of Data":"Gender"}).sort_values(by = "Proportion of Days Eaten before Covid", ascending = False)
eat_pre_prop_sharks = props_pre.iloc[0:12,:].rename(columns = {"Subset of Data":"Shark"}).sort_values(by = "Proportion of Days Eaten before Covid", ascending = False)

#Proportions during covid
eat_covid_prop_species = props_covid.iloc[15:,:].rename(columns = {"Subset of Data":"Species"}).sort_values(by = "Proportion of Days Eaten during Shutdown", ascending = False)
eat_covid_prop_gender = props_covid.iloc[13:15,:].rename(columns = {"Subset of Data":"Gender"}).sort_values(by = "Proportion of Days Eaten during Shutdown", ascending = False)
eat_covid_prop_sharks = props_covid.iloc[0:12,:].rename(columns = {"Subset of Data":"Shark"}).sort_values(by = "Proportion of Days Eaten during Shutdown", ascending = False)

#Proportions after covid
eat_post_prop_species = props_post.iloc[15:,:].rename(columns = {"Subset of Data":"Species"}).sort_values(by = "Proportion of Days Eaten after Reopening", ascending = False)
eat_post_prop_gender = props_post.iloc[13:15,:].rename(columns = {"Subset of Data":"Gender"}).sort_values(by = "Proportion of Days Eaten after Reopening", ascending = False)
eat_post_prop_sharks = props_post.iloc[0:12,:].rename(columns = {"Subset of Data":"Shark"}).sort_values(by = "Proportion of Days Eaten after Reopening", ascending = False)

In [396]:
px.bar(eat_pre_prop_species, x = "Species", y = "Proportion of Days Eaten before Covid", title = "What proportion of days does each species eat before covid?")

In [397]:
px.bar(eat_covid_prop_species, x = "Species", y = "Proportion of Days Eaten during Shutdown", title = "What proportion of days does each species eat during Shutdown?")

In [398]:
px.bar(eat_post_prop_species, x = "Species", y = "Proportion of Days Eaten after Reopening", title = "What proportion of days does each species eat after Reopening?")

In [399]:
px.bar(eat_pre_prop_gender, x = "Gender", y = "Proportion of Days Eaten before Covid", title = "What proportion of days does each gender eat before covid?")

In [400]:
px.bar(eat_covid_prop_gender, x = "Gender", y = "Proportion of Days Eaten during Shutdown", title = "What proportion of days does each gender eat during Shutdown?")

In [401]:
px.bar(eat_post_prop_gender, x = "Gender", y = "Proportion of Days Eaten after Reopening", title = "What proportion of days does each gender eat after Reopening?")

In [402]:
px.bar(eat_pre_prop_sharks, x = "Shark", y = "Proportion of Days Eaten before Covid", title = "What proportion of days does each shark eat before covid?")

In [403]:
px.bar(eat_covid_prop_sharks, x = "Shark", y = "Proportion of Days Eaten during Shutdown", title = "What proportion of days does each shark eat during the Shutdown?")

In [404]:
px.bar(eat_post_prop_sharks, x = "Shark", y = "Proportion of Days Eaten after Reopening", title = "What proportion of days does each shark eat after Reopening?")

# Targeting

In [405]:
all_sharks_pre = targets_pre[['Ross', 'Chandler','BT1','BT2','BT3','BT4','BT5','GR1','GR2','GR3','GR4', 'GR5']]
all_sharks_covid = targets_covid[['Ross', 'Chandler','BT1','BT2','BT3','BT4','BT5','GR1','GR2','GR3','GR4', 'GR5']]
all_sharks_post = targets_post[['Ross', 'Chandler','BT1','BT2','BT3','BT4','BT5','GR1','GR2','GR3','GR4', 'GR5']]

In [406]:
all_mean_pre = round(all_sharks_pre.mean(), 3)
all_mean_covid = round(all_sharks_covid.mean(), 3)
all_mean_post = round(all_sharks_post.mean(), 3)
all_mean_pre

Ross        10.071
Chandler     9.436
BT1          9.989
BT2          7.114
BT3          7.254
BT4          7.969
BT5          6.795
GR1          2.568
GR2          3.310
GR3          5.148
GR4          2.852
GR5          3.838
dtype: float64

In [407]:
all_mean_covid

Ross        6.652
Chandler    7.000
BT1         7.435
BT2         8.478
BT3         7.826
BT4         8.087
BT5         5.000
GR1         1.571
GR2         2.857
GR3         4.190
GR4         2.857
GR5         2.476
dtype: float64

In [408]:
all_mean_post

Ross        7.758
Chandler    7.726
BT1         5.452
BT2         6.145
BT3         6.355
BT4         6.387
BT5         6.419
GR1         2.459
GR2         2.672
GR3         3.541
GR4         2.443
GR5         3.607
dtype: float64

In [409]:
# Graph Mean Number of Fish Targeted For Each Shark for each time pre, during, post covid.
targeted_means = {'Shark': all_sharks_pre.columns,
               'Pre_Mean': all_mean_pre,
               'Covid_Mean': all_mean_covid,
               'Post_Mean': all_mean_post}

targeted_means_df = pd.DataFrame(targeted_means, columns = ['Shark', 'Pre_Mean', 'Covid_Mean','Post_Mean'])


pre_targeted_means_sorted = targeted_means_df.sort_values(by=['Pre_Mean'], ascending = False)
covid_targeted_means_sorted = targeted_means_df.sort_values(by=['Covid_Mean'], ascending = False)
post_targeted_means_sorted = targeted_means_df.sort_values(by=['Post_Mean'], ascending = False)


fig = px.histogram(pre_targeted_means_sorted, x="Shark", y="Pre_Mean", 
                   title = "Mean Number of Fish Targeted by Shark before Shutdown",
                  labels={
                     "Shark": "Shark",
                     "Mean": " Mean Number of Fish",
                 })
fig.show()

In [410]:
# Shark targeting vector pre covid
total_sharks_targeting_pre = targets_pre[["Ross","Chandler","BT1","BT2","BT3","BT4","BT5","GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
BT_sharks_targeting_pre = targets_pre[["BT1","BT2","BT3","BT4","BT5"]].count(axis = 1)
GR_sharks_targeting_pre = targets_pre[["GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
SS_sharks_targeting_pre = targets_pre[["Ross","Chandler"]].count(axis = 1)
male_sharks_targeting_pre = targets_pre[["Ross","Chandler","BT1","BT5","GR1"]].count(axis = 1)
female_sharks_targeting_pre = targets_pre[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].count(axis = 1)

# Shark targeting vector during covid
total_sharks_targeting_covid = targets_covid[["Ross","Chandler","BT1","BT2","BT3","BT4","BT5","GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
BT_sharks_targeting_covid = targets_covid[["BT1","BT2","BT3","BT4","BT5"]].count(axis = 1)
GR_sharks_targeting_covid = targets_covid[["GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
SS_sharks_targeting_covid = targets_covid[["Ross","Chandler"]].count(axis = 1)
male_sharks_targeting_covid = targets_covid[["Ross","Chandler","BT1","BT5","GR1"]].count(axis = 1)
female_sharks_targeting_covid = targets_covid[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].count(axis = 1)

# Shark targeting vector post covid
total_sharks_targeting_post = targets_post[["Ross","Chandler","BT1","BT2","BT3","BT4","BT5","GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
BT_sharks_targeting_post = targets_post[["BT1","BT2","BT3","BT4","BT5"]].count(axis = 1)
GR_sharks_targeting_post = targets_post[["GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
SS_sharks_targeting_post = targets_post[["Ross","Chandler"]].count(axis = 1)
male_sharks_targeting_post = targets_post[["Ross","Chandler","BT1","BT5","GR1"]].count(axis = 1)
female_sharks_targeting_post = targets_post[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].count(axis = 1)
total_sharks_targeting_post


501     7
502    12
503    12
504     5
505     7
       ..
578    12
579    12
580     5
581     7
582    12
Length: 82, dtype: int64

In [411]:
targets_pre["male"] = targets_pre[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
targets_pre["female"] = targets_pre[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)
targets_covid["male"] = targets_covid[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
targets_covid["female"] = targets_covid[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)
targets_post["male"] = targets_post[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
targets_post["female"] = targets_post[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)

targets_pre["All_GR"] = targets_pre["GR1"] + targets_pre["GR2"] + targets_pre["GR3"]+ targets_pre["GR4"] + targets_pre["GR5"]
targets_pre["All_BT"] = targets_pre["BT1"] + targets_pre["BT2"] + targets_pre["BT3"]+ targets_pre["BT4"] + targets_pre["BT5"]
targets_pre["All_SS"] = targets_pre["Ross"] + targets_pre["Chandler"]

targets_covid["All_GR"] = targets_covid["GR1"] + targets_covid["GR2"] + targets_covid["GR3"]+ targets_covid["GR4"] + targets_covid["GR5"]
targets_covid["All_BT"] = targets_covid["BT1"] + targets_covid["BT2"] + targets_covid["BT3"]+ targets_covid["BT4"] + targets_covid["BT5"]
targets_covid["All_SS"] = targets_covid["Ross"] + targets_covid["Chandler"]

targets_post["All_GR"] = targets_post["GR1"] + targets_post["GR2"] + targets_post["GR3"]+ targets_post["GR4"] + targets_post["GR5"]
targets_post["All_BT"] = targets_post["BT1"] + targets_post["BT2"] + targets_post["BT3"]+ targets_post["BT4"] + targets_post["BT5"]
targets_post["All_SS"] = targets_post["Ross"] + targets_post["Chandler"]


<ipython-input-411-2fa4ba29c82c>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-411-2fa4ba29c82c>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-411-2fa4ba29c82c>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-411-2fa

In [412]:
#Average Mean Graphs Gender - Pre

all_sharks_targets_pre = targets_pre[["female","male"]]
all_sharks_targets_pre["female"] = all_sharks_targets_pre["female"] / female_sharks_feeding_pre
all_sharks_targets_pre["male"] = all_sharks_targets_pre["male"] / male_sharks_feeding_pre
targets_pre_means = pd.DataFrame(round(all_sharks_targets_pre.mean(), 3)).reset_index()
targets_pre_means.columns = ["Gender","Mean"]
targets_pre_means = targets_pre_means.sort_values(by = "Mean", ascending = False)
print(targets_pre_means) 
px.bar(targets_pre_means, x = "Gender", y = "Mean", title = "How often is the average shark targeting for each gender before Covid?")

   Gender   Mean
1    male  7.010
0  female  5.712


<ipython-input-412-9a7a3bc87e0e>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-412-9a7a3bc87e0e>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [413]:
#Average Mean Graphs Gender - During Covid

all_sharks_targets_covid = targets_covid[["female","male"]]
all_sharks_targets_covid["female"] = all_sharks_targets_covid["female"] / female_sharks_feeding_covid
all_sharks_targets_covid["male"] = all_sharks_targets_covid["male"] / male_sharks_feeding_covid
targets_covid_means = pd.DataFrame(round(all_sharks_targets_covid.mean(), 3)).reset_index()
targets_covid_means.columns = ["Gender","Mean"]
targets_covid_means = targets_covid_means.sort_values(by = "Mean", ascending = False)
print(targets_covid_means) 
px.bar(targets_covid_means, x = "Gender", y = "Mean", title = "How often is the average shark targeting for each gender without patrons?")


   Gender   Mean
0  female  5.744
1    male  5.204


<ipython-input-413-de46b8a0836a>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-413-de46b8a0836a>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [414]:
#Average Mean Graphs Gender - Post

all_sharks_targets_post = targets_post[["female","male"]]
all_sharks_targets_post["female"] = all_sharks_targets_post["female"] / female_sharks_feeding_post
all_sharks_targets_post["male"] = all_sharks_targets_post["male"] / male_sharks_feeding_post
targets_post_means = pd.DataFrame(round(all_sharks_targets_post.mean(), 3)).reset_index()
targets_post_means.columns = ["Gender","Mean"]
targets_post_means = targets_post_means.sort_values(by = "Mean", ascending = False)
print(targets_post_means) 
px.bar(targets_post_means, x = "Gender", y = "Mean", title = "How often is the average shark targeting for each gender after Reopening")

<ipython-input-414-42f0bc3d199f>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-414-42f0bc3d199f>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



   Gender   Mean
1    male  5.354
0  female  4.691


In [415]:
#Average Mean Graphs Per Species - Pre

all_sharks_targets_pre = targets_pre[["All_GR","All_BT","All_SS"]]
all_sharks_targets_pre["All_GR"] = all_sharks_targets_pre["All_GR"] / GR_sharks_feeding_pre
all_sharks_targets_pre["All_BT"] = all_sharks_targets_pre["All_BT"] / BT_sharks_feeding_pre
all_sharks_targets_pre["All_SS"] = all_sharks_targets_pre["All_SS"] / SS_sharks_feeding_pre
all_sharks_targets_pre.columns = ["Grey Reef","Black Tip","Sand Shark"]
targets_pre_means = pd.DataFrame(round(all_sharks_targets_pre.mean(), 3)).reset_index()
targets_pre_means.columns = ["Species","Mean"]
targets_pre_means = targets_pre_means.sort_values(by = "Mean", ascending = False)
print(targets_pre_means)
px.bar(targets_pre_means, x = "Species", y = "Mean", title = "Target Means for Each species on average before covid")


<ipython-input-415-6a41ca69247a>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-415-6a41ca69247a>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-415-6a41ca69247a>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



      Species   Mean
2  Sand Shark  9.754
1   Black Tip  7.834
0   Grey Reef  3.543


In [416]:
#Average Mean Graphs Per Species - During

all_sharks_targets_covid = targets_covid[["All_GR","All_BT","All_SS"]]
all_sharks_targets_covid["All_GR"] = all_sharks_targets_covid["All_GR"] / GR_sharks_feeding_covid
all_sharks_targets_covid["All_BT"] = all_sharks_targets_covid["All_BT"] / BT_sharks_feeding_covid
all_sharks_targets_covid["All_SS"] = all_sharks_targets_covid["All_SS"] / SS_sharks_feeding_covid
all_sharks_targets_covid.columns = ["Grey Reef","Black Tip","Sand Shark"]
targets_covid_means = pd.DataFrame(round(all_sharks_targets_covid.mean(), 3)).reset_index()
targets_covid_means.columns = ["Species","Mean"]
targets_covid_means = targets_covid_means.sort_values(by = "Mean", ascending = False)
print(targets_covid_means)
px.bar(targets_covid_means, x = "Species", y = "Mean", title = "Target Means for Each species on average during shutdown")


<ipython-input-416-f75b19ae98a7>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-416-f75b19ae98a7>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-416-f75b19ae98a7>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



      Species   Mean
1   Black Tip  7.365
2  Sand Shark  6.826
0   Grey Reef  2.790


In [417]:
#Average Mean Graphs Per Species post - Post

all_sharks_targets_post = targets_post[["All_GR","All_BT","All_SS"]]
all_sharks_targets_post["All_GR"] = all_sharks_targets_post["All_GR"] / GR_sharks_feeding_post
all_sharks_targets_post["All_BT"] = all_sharks_targets_post["All_BT"] / BT_sharks_feeding_post
all_sharks_targets_post["All_SS"] = all_sharks_targets_post["All_SS"] / SS_sharks_feeding_post
all_sharks_targets_post.columns = ["Grey Reef","Black Tip","Sand Shark"]
targets_post_means = pd.DataFrame(round(all_sharks_targets_post.mean(), 3)).reset_index()
targets_post_means.columns = ["Species","Mean"]
targets_post_means = targets_post_means.sort_values(by = "Mean", ascending = False)
print(targets_post_means)
px.bar(targets_post_means, x = "Species", y = "Mean", title = "Target Means for Each species on average after reopening")


      Species   Mean
2  Sand Shark  7.742
1   Black Tip  6.152
0   Grey Reef  2.944


<ipython-input-417-f6a239a7c71c>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-417-f6a239a7c71c>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-417-f6a239a7c71c>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [418]:
All_Sharks_targets_pre = targets_pre.iloc[:,1:]
All_Sharks_targets_pre["All_GR"] = All_Sharks_targets_pre["All_GR"] / GR_sharks_feeding_pre
All_Sharks_targets_pre["All_BT"] = All_Sharks_targets_pre["All_BT"] / BT_sharks_feeding_pre
All_Sharks_targets_pre["All_SS"] = All_Sharks_targets_pre["All_SS"] / SS_sharks_feeding_pre
All_Sharks_targets_pre["female"] = All_Sharks_targets_pre["female"] / female_sharks_feeding_pre
All_Sharks_targets_pre["male"] = All_Sharks_targets_pre["male"] / male_sharks_feeding_pre
All_Sharks_targets_pre["Total: "] = All_Sharks_targets_pre["Total: "] / total_sharks_feeding_pre
targets_pre_means = pd.DataFrame(round(All_Sharks_targets_pre.mean(), 3)).reset_index()
targets_pre_means.columns = ["Subset","Mean"]
targets_pre_means = targets_pre_means.sort_values(by = "Mean", ascending = False)
ci = {}
for column in targets_pre.iloc[:,1:].columns:
    conf = st.t.interval(alpha=0.95, df=len(targets_pre[column])-1, loc=np.mean(targets_pre[column]), scale=st.sem(targets_pre[column],nan_policy='omit'))
    error = (conf[1]-conf[0])/2
    ci.update({column:error})
ci = pd.DataFrame(list(ci.items()),columns = ['Subset','SE'])
targets_pre_means = pd.merge(targets_pre_means, ci, how = 'outer', on = "Subset")
#targets_pre_means = pd.merge(targets_pre_means,pd.melt(targets_pre.describe().loc[["std"]], var_name = "Subset",value_name = "Standard Deviation"), how = "outer", on = "Subset")
prop_average = targets_pre_means[targets_pre_means["Subset"]=="Total: "]["Mean"].values[0]
Average = []
for i in targets_pre_means.Mean:
    if i > prop_average:
        Average.append("Above Average")
    elif i == prop_average:
        Average.append("Average")
    else:
        Average.append("Below Average")
targets_pre_means["Average"] = Average
max_y = (targets_pre_means["Mean"] + targets_pre_means["SE"]).max() 
#max_y = (targets_pre_means["Mean"] + targets_pre_means["Standard Deviation"]).max() 
px.bar(targets_pre_means, x = "Subset", y = "Mean", color = "Average",text = "Mean", error_y = "SE",title = "target Means for each subset with Confidence Intervals before covid").update_traces(textposition='outside')


In [419]:
all_sharks_targets_covid = targets_covid.iloc[:,1:]
all_sharks_targets_covid["All_GR"] = all_sharks_targets_covid["All_GR"] / GR_sharks_feeding_covid
all_sharks_targets_covid["All_BT"] = all_sharks_targets_covid["All_BT"] / BT_sharks_feeding_covid
all_sharks_targets_covid["All_SS"] = all_sharks_targets_covid["All_SS"] / SS_sharks_feeding_covid
all_sharks_targets_covid["female"] = all_sharks_targets_covid["female"] / female_sharks_feeding_covid
all_sharks_targets_covid["male"] = all_sharks_targets_covid["male"] / male_sharks_feeding_covid
all_sharks_targets_covid["Total: "] = all_sharks_targets_covid["Total: "] / total_sharks_feeding_covid
targets_covid_means = pd.DataFrame(round(all_sharks_targets_covid.mean(), 3)).reset_index()
targets_covid_means.columns = ["Subset","Mean"]
targets_covid_means = targets_covid_means.sort_values(by = "Mean", ascending = False)
ci = {}
for column in targets_covid.iloc[:,1:].columns:
    conf = st.t.interval(alpha=0.95, df=len(targets_covid[column])-1, loc=np.mean(targets_covid[column]), scale=st.sem(targets_covid[column],nan_policy='omit'))
    error = (conf[1]-conf[0])/2
    ci.update({column:error})
ci = pd.DataFrame(list(ci.items()),columns = ['Subset','SE'])
targets_covid_means = pd.merge(targets_covid_means, ci, how = 'outer', on = "Subset")
#targets_covid_means = pd.merge(targets_covid_means,pd.melt(targets_covid.describe().loc[["std"]], var_name = "Subset",value_name = "Standard Deviation"), how = "outer", on = "Subset")
prop_average = targets_covid_means[targets_covid_means["Subset"]=="Total: "]["Mean"].values[0]
Average = []
for i in targets_covid_means.Mean:
    if i > prop_average:
        Average.append("Above Average")
    elif i == prop_average:
        Average.append("Average")
    else:
        Average.append("Below Average")
targets_covid_means["Average"] = Average
max_y = (targets_covid_means["Mean"] + targets_covid_means["SE"]).max() 
#max_y = (targets_covid_means["Mean"] + targets_covid_means["Standard Deviation"]).max() 
px.bar(targets_covid_means, x = "Subset", y = "Mean", color = "Average",text = "Mean", error_y = "SE",title = "Target Means for each subset with Confidence Intervals during shutdown").update_traces(textposition='outside')


In [420]:
All_Sharks_targets_post = targets_post.iloc[:,1:]
All_Sharks_targets_post["All_GR"] = All_Sharks_targets_post["All_GR"] / GR_sharks_feeding_post
All_Sharks_targets_post["All_BT"] = All_Sharks_targets_post["All_BT"] / BT_sharks_feeding_post
All_Sharks_targets_post["All_SS"] = All_Sharks_targets_post["All_SS"] / SS_sharks_feeding_post
All_Sharks_targets_post["female"] = All_Sharks_targets_post["female"] / female_sharks_feeding_post
All_Sharks_targets_post["male"] = All_Sharks_targets_post["male"] / male_sharks_feeding_post
All_Sharks_targets_post["Total: "] = All_Sharks_targets_post["Total: "] / total_sharks_feeding_post
targets_post_means = pd.DataFrame(round(All_Sharks_targets_post.mean(), 3)).reset_index()
targets_post_means.columns = ["Subset","Mean"]
targets_post_means = targets_post_means.sort_values(by = "Mean", ascending = False)
ci = {}
for column in targets_post.iloc[:,1:].columns:
    conf = st.t.interval(alpha=0.95, df=len(targets_post[column])-1, loc=np.mean(targets_post[column]), scale=st.sem(targets_post[column],nan_policy='omit'))
    error = (conf[1]-conf[0])/2
    ci.update({column:error})
ci = pd.DataFrame(list(ci.items()),columns = ['Subset','SE'])
targets_post_means = pd.merge(targets_post_means, ci, how = 'outer', on = "Subset")
#targets_post_means = pd.merge(targets_post_means,pd.melt(targets_post.describe().loc[["std"]], var_name = "Subset",value_name = "Standard Deviation"), how = "outer", on = "Subset")
prop_average = targets_post_means[targets_post_means["Subset"]=="Total: "]["Mean"].values[0]
Average = []
for i in targets_post_means.Mean:
    if i > prop_average:
        Average.append("Above Average")
    elif i == prop_average:
        Average.append("Average")
    else:
        Average.append("Below Average")
targets_post_means["Average"] = Average
max_y = (targets_post_means["Mean"] + targets_post_means["SE"]).max() 
#max_y = (targets_post_means["Mean"] + targets_post_means["Standard Deviation"]).max() 
px.bar(targets_post_means, x = "Subset", y = "Mean", color = "Average",text = "Mean", error_y = "SE",title = "Target Means for each subset with Confidence Intervals after Shutdown").update_traces(textposition='outside')

In [421]:
targets_pre_prop = targets_pre.iloc[:,1:]
targets_covid_prop = targets_covid.iloc[:,1:]
targets_post_prop = targets_post.iloc[:,1:]


props_pre = {}
for i in targets_pre_prop.columns:
    data = targets_pre_prop[i]
    targets_pre_days = data > 0
    props_pre.update({i:(round(len(data[targets_pre_days])/len(data[data.isna()==False]),2))})
props_pre = pd.DataFrame(list(props_pre.items()),columns = ['Subset of Data','Proportion of Days targeted before Covid'])


props_covid = {}
for i in targets_pre_prop.columns:
    data = targets_pre_prop[i]
    targets_pre_days = data > 0
    props_covid.update({i:(round(len(data[targets_pre_days])/len(data[data.isna()==False]),2))})
props_covid = pd.DataFrame(list(props_covid.items()),columns = ['Subset of Data','Proportion of Days targeted during Shutdown'])


props_post = {}
for i in targets_pre_prop.columns:
    data = targets_pre_prop[i]
    targets_pre_days = data > 0
    props_post.update({i:(round(len(data[targets_pre_days])/len(data[data.isna()==False]),2))})
props_post = pd.DataFrame(list(props_post.items()),columns = ['Subset of Data','Proportion of Days targeted after Reopening'])


print(props_pre.iloc[12].values[1],"is the overall proportion of days where a shark ate before covid")
print(props_covid.iloc[12].values[1],"is the overall proportion of days where a shark ate during shutdown")
print(props_post.iloc[12].values[1],"is the overall proportion of days where a shark ate after aquarium reopened")

1.0 is the overall proportion of days where a shark ate before covid
1.0 is the overall proportion of days where a shark ate during shutdown
1.0 is the overall proportion of days where a shark ate after aquarium reopened


In [422]:
# Proportion before covid
targets_pre_prop_species = props_pre.iloc[15:,:].rename(columns = {"Subset of Data":"Species"}).sort_values(by = "Proportion of Days targeted before Covid", ascending = False)
targets_pre_prop_gender = props_pre.iloc[13:15,:].rename(columns = {"Subset of Data":"Gender"}).sort_values(by = "Proportion of Days targeted before Covid", ascending = False)
targets_pre_prop_sharks = props_pre.iloc[0:12,:].rename(columns = {"Subset of Data":"Shark"}).sort_values(by = "Proportion of Days targeted before Covid", ascending = False)

#Proportions during covid
targets_covid_prop_species = props_covid.iloc[15:,:].rename(columns = {"Subset of Data":"Species"}).sort_values(by = "Proportion of Days targeted during Shutdown", ascending = False)
targets_covid_prop_gender = props_covid.iloc[13:15,:].rename(columns = {"Subset of Data":"Gender"}).sort_values(by = "Proportion of Days targeted during Shutdown", ascending = False)
targets_covid_prop_sharks = props_covid.iloc[0:12,:].rename(columns = {"Subset of Data":"Shark"}).sort_values(by = "Proportion of Days targeted during Shutdown", ascending = False)

#Proportions after covid
targets_post_prop_species = props_post.iloc[15:,:].rename(columns = {"Subset of Data":"Species"}).sort_values(by = "Proportion of Days targeted after Reopening", ascending = False)
targets_post_prop_gender = props_post.iloc[13:15,:].rename(columns = {"Subset of Data":"Gender"}).sort_values(by = "Proportion of Days targeted after Reopening", ascending = False)
targets_post_prop_sharks = props_post.iloc[0:12,:].rename(columns = {"Subset of Data":"Shark"}).sort_values(by = "Proportion of Days targeted after Reopening", ascending = False)


In [423]:
px.bar(targets_pre_prop_species, x = "Species", y = "Proportion of Days targeted before Covid", title = "What proportion of days does each species target before covid?")

In [424]:
px.bar(targets_covid_prop_species, x = "Species", y = "Proportion of Days targeted during Shutdown", title = "What proportion of days does each species target during Shutdown?")

In [425]:
px.bar(targets_post_prop_species, x = "Species", y = "Proportion of Days targeted after Reopening", title = "What proportion of days does each species target after Reopening?")

In [426]:
px.bar(targets_pre_prop_gender, x = "Gender", y = "Proportion of Days targeted before Covid", title = "What proportion of days does each gender target before covid?")

In [427]:
px.bar(targets_covid_prop_gender, x = "Gender", y = "Proportion of Days targeted during Shutdown", title = "What proportion of days does each gender target during Shutdown?")

In [428]:
px.bar(targets_post_prop_gender, x = "Gender", y = "Proportion of Days targeted after Reopening", title = "What proportion of days does each gender target after Reopening?")

In [429]:
px.bar(targets_pre_prop_sharks, x = "Shark", y = "Proportion of Days targeted before Covid", title = "What proportion of days does each shark target before covid?")

In [430]:
px.bar(targets_covid_prop_sharks, x = "Shark", y = "Proportion of Days targeted during Shutdown", title = "What proportion of days does each shark target during the Shutdown?")

In [431]:
px.bar(targets_post_prop_sharks, x = "Shark", y = "Proportion of Days targeted after Reopening", title = "What proportion of days does each shark target after Reopening?")

# Drops

In [432]:
all_sharks_pre = drops_pre[['Ross', 'Chandler','BT1','BT2','BT3','BT4','BT5','GR1','GR2','GR3','GR4', 'GR5']]
all_sharks_covid = drops_covid[['Ross', 'Chandler','BT1','BT2','BT3','BT4','BT5','GR1','GR2','GR3','GR4', 'GR5']]
all_sharks_post = drops_post[['Ross', 'Chandler','BT1','BT2','BT3','BT4','BT5','GR1','GR2','GR3','GR4', 'GR5']]

In [433]:
all_mean_pre = round(all_sharks_pre.mean(), 3)
all_mean_covid = round(all_sharks_covid.mean(), 3)
all_mean_post = round(all_sharks_post.mean(), 3)
all_mean_pre

Ross        0.139
Chandler    0.239
BT1         0.815
BT2         0.841
BT3         0.543
BT4         0.474
BT5         0.557
GR1         0.116
GR2         0.249
GR3         0.283
GR4         0.161
GR5         0.099
dtype: float64

In [434]:
all_mean_covid

Ross        0.261
Chandler    0.304
BT1         0.826
BT2         1.783
BT3         0.913
BT4         0.826
BT5         1.190
GR1         0.095
GR2         0.333
GR3         0.381
GR4         0.190
GR5         0.143
dtype: float64

In [435]:
all_mean_post

Ross        0.065
Chandler    0.323
BT1         0.516
BT2         0.661
BT3         0.774
BT4         0.355
BT5         0.677
GR1         0.082
GR2         0.295
GR3         0.262
GR4         0.115
GR5         0.230
dtype: float64

In [436]:
# Graph Mean Number of Fish dropped For Each Shark for each time pre, during, post covid.
dropped_means = {'Shark': all_sharks_pre.columns,
               'Pre_Mean': all_mean_pre,
               'Covid_Mean': all_mean_covid,
               'Post_Mean': all_mean_post}

In [437]:
dropped_means_df = pd.DataFrame(dropped_means, columns = ['Shark', 'Pre_Mean', 'Covid_Mean','Post_Mean'])

In [438]:
pre_dropped_means_sorted = dropped_means_df.sort_values(by=['Pre_Mean'], ascending = False)
covid_dropped_means_sorted = dropped_means_df.sort_values(by=['Covid_Mean'], ascending = False)
post_dropped_means_sorted = dropped_means_df.sort_values(by=['Post_Mean'], ascending = False)

In [439]:
fig = px.histogram(pre_dropped_means_sorted, x="Shark", y="Pre_Mean", 
                   title = "Mean Number of Fish dropped by Shark before Shutdown",
                  labels={
                     "Shark": "Shark",
                     "Mean": " Mean Number of Fish",
                 })
fig.show()

In [440]:
# Shark dropping vector pre covid
total_sharks_dropping_pre = drops_pre[["Ross","Chandler","BT1","BT2","BT3","BT4","BT5","GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
BT_sharks_dropping_pre = drops_pre[["BT1","BT2","BT3","BT4","BT5"]].count(axis = 1)
GR_sharks_dropping_pre = drops_pre[["GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
SS_sharks_dropping_pre = drops_pre[["Ross","Chandler"]].count(axis = 1)
male_sharks_dropping_pre = drops_pre[["Ross","Chandler","BT1","BT5","GR1"]].count(axis = 1)
female_sharks_dropping_pre = drops_pre[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].count(axis = 1)

# Shark dropping vector during covid
total_sharks_dropping_covid = drops_covid[["Ross","Chandler","BT1","BT2","BT3","BT4","BT5","GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
BT_sharks_dropping_covid = drops_covid[["BT1","BT2","BT3","BT4","BT5"]].count(axis = 1)
GR_sharks_dropping_covid = drops_covid[["GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
SS_sharks_dropping_covid = drops_covid[["Ross","Chandler"]].count(axis = 1)
male_sharks_dropping_covid = drops_covid[["Ross","Chandler","BT1","BT5","GR1"]].count(axis = 1)
female_sharks_dropping_covid = drops_covid[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].count(axis = 1)

# Shark dropping vector post covid
total_sharks_dropping_post = drops_post[["Ross","Chandler","BT1","BT2","BT3","BT4","BT5","GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
BT_sharks_dropping_post = drops_post[["BT1","BT2","BT3","BT4","BT5"]].count(axis = 1)
GR_sharks_dropping_post = drops_post[["GR1","GR2","GR3","GR4","GR5"]].count(axis = 1)
SS_sharks_dropping_post = drops_post[["Ross","Chandler"]].count(axis = 1)
male_sharks_dropping_post = drops_post[["Ross","Chandler","BT1","BT5","GR1"]].count(axis = 1)
female_sharks_dropping_post = drops_post[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].count(axis = 1)
total_sharks_dropping_post


501     7
502    12
503    12
504     5
505     7
       ..
578    12
579    12
580     5
581     7
582    12
Length: 82, dtype: int64

In [441]:
drops_pre["male"] = drops_pre[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
drops_pre["female"] = drops_pre[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)
drops_covid["male"] = drops_covid[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
drops_covid["female"] = drops_covid[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)
drops_post["male"] = drops_post[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
drops_post["female"] = drops_post[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)

drops_pre["All_GR"] = drops_pre["GR1"] + drops_pre["GR2"] + drops_pre["GR3"]+ drops_pre["GR4"] + drops_pre["GR5"]
drops_pre["All_BT"] = drops_pre["BT1"] + drops_pre["BT2"] + drops_pre["BT3"]+ drops_pre["BT4"] + drops_pre["BT5"]
drops_pre["All_SS"] = drops_pre["Ross"] + drops_pre["Chandler"]

drops_covid["All_GR"] = drops_covid["GR1"] + drops_covid["GR2"] + drops_covid["GR3"]+ drops_covid["GR4"] + drops_covid["GR5"]
drops_covid["All_BT"] = drops_covid["BT1"] + drops_covid["BT2"] + drops_covid["BT3"]+ drops_covid["BT4"] + drops_covid["BT5"]
drops_covid["All_SS"] = drops_covid["Ross"] + drops_covid["Chandler"]

drops_post["All_GR"] = drops_post["GR1"] + drops_post["GR2"] + drops_post["GR3"]+ drops_post["GR4"] + drops_post["GR5"]
drops_post["All_BT"] = drops_post["BT1"] + drops_post["BT2"] + drops_post["BT3"]+ drops_post["BT4"] + drops_post["BT5"]
drops_post["All_SS"] = drops_post["Ross"] + drops_post["Chandler"]

<ipython-input-441-64c7e049d6ad>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-441-64c7e049d6ad>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-441-64c7e049d6ad>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-441-64c

In [442]:
#Average Mean Graphs Gender - Pre

all_sharks_drops_pre = drops_pre[["female","male"]]
all_sharks_drops_pre["female"] = all_sharks_drops_pre["female"] / female_sharks_feeding_pre
all_sharks_drops_pre["male"] = all_sharks_drops_pre["male"] / male_sharks_feeding_pre
drops_pre_means = pd.DataFrame(round(all_sharks_drops_pre.mean(), 3)).reset_index()
drops_pre_means.columns = ["Gender","Mean"]
drops_pre_means = drops_pre_means.sort_values(by = "Mean", ascending = False)
print(drops_pre_means) 
px.bar(drops_pre_means, x = "Gender", y = "Mean", title = "How often is the average shark dropping for each gender before Covid?")

   Gender   Mean
0  female  0.403
1    male  0.334


<ipython-input-442-3c40ea1c0635>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-442-3c40ea1c0635>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [443]:
#Average Mean Graphs Gender - During Covid

all_sharks_drops_covid = drops_covid[["female","male"]]
all_sharks_drops_covid["female"] = all_sharks_drops_covid["female"] / female_sharks_feeding_covid
all_sharks_drops_covid["male"] = all_sharks_drops_covid["male"] / male_sharks_feeding_covid
drops_covid_means = pd.DataFrame(round(all_sharks_drops_covid.mean(), 3)).reset_index()
drops_covid_means.columns = ["Gender","Mean"]
drops_covid_means = drops_covid_means.sort_values(by = "Mean", ascending = False)
print(drops_covid_means) 
px.bar(drops_covid_means, x = "Gender", y = "Mean", title = "How often is the average shark dropping for each gender without patrons?")

   Gender   Mean
0  female  0.700
1    male  0.439


<ipython-input-443-54317ef29142>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-443-54317ef29142>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [444]:
#Average Mean Graphs Gender - Post

all_sharks_drops_post = drops_post[["female","male"]]
all_sharks_drops_post["female"] = all_sharks_drops_post["female"] / female_sharks_feeding_post
all_sharks_drops_post["male"] = all_sharks_drops_post["male"] / male_sharks_feeding_post
drops_post_means = pd.DataFrame(round(all_sharks_drops_post.mean(), 3)).reset_index()
drops_post_means.columns = ["Gender","Mean"]
drops_post_means = drops_post_means.sort_values(by = "Mean", ascending = False)
print(drops_post_means) 
px.bar(drops_post_means, x = "Gender", y = "Mean", title = "How often is the average shark dropping for each gender after shutdwon")

   Gender   Mean
0  female  0.357
1    male  0.286


<ipython-input-444-70dbfa0bed61>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-444-70dbfa0bed61>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [445]:
#Average Mean Graphs Per Species - Pre

all_sharks_drops_pre = drops_pre[["All_GR","All_BT","All_SS"]]
all_sharks_drops_pre["All_GR"] = all_sharks_drops_pre["All_GR"] / GR_sharks_feeding_pre
all_sharks_drops_pre["All_BT"] = all_sharks_drops_pre["All_BT"] / BT_sharks_feeding_pre
all_sharks_drops_pre["All_SS"] = all_sharks_drops_pre["All_SS"] / SS_sharks_feeding_pre
all_sharks_drops_pre.columns = ["Grey Reef","Black Tip","Sand Shark"]
drops_pre_means = pd.DataFrame(round(all_sharks_drops_pre.mean(), 3)).reset_index()
drops_pre_means.columns = ["Species","Mean"]
drops_pre_means = drops_pre_means.sort_values(by = "Mean", ascending = False)
print(drops_pre_means)
px.bar(drops_pre_means, x = "Species", y = "Mean", title = "Drop Means for Each species on average before covid")


<ipython-input-445-00972cf247d8>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-445-00972cf247d8>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-445-00972cf247d8>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



      Species   Mean
1   Black Tip  0.646
2  Sand Shark  0.189
0   Grey Reef  0.182


In [446]:
#Average Mean Graphs Per Species - During

all_sharks_drops_covid = drops_covid[["All_GR","All_BT","All_SS"]]
all_sharks_drops_covid["All_GR"] = all_sharks_drops_covid["All_GR"] / GR_sharks_feeding_covid
all_sharks_drops_covid["All_BT"] = all_sharks_drops_covid["All_BT"] / BT_sharks_feeding_covid
all_sharks_drops_covid["All_SS"] = all_sharks_drops_covid["All_SS"] / SS_sharks_feeding_covid
all_sharks_drops_covid.columns = ["Grey Reef","Black Tip","Sand Shark"]
drops_covid_means = pd.DataFrame(round(all_sharks_drops_covid.mean(), 3)).reset_index()
drops_covid_means.columns = ["Species","Mean"]
drops_covid_means = drops_covid_means.sort_values(by = "Mean", ascending = False)
print(drops_covid_means)
px.bar(drops_covid_means, x = "Species", y = "Mean", title = "Drop Means for Each species on average during shutdown")

      Species   Mean
1   Black Tip  1.114
2  Sand Shark  0.283
0   Grey Reef  0.229


<ipython-input-446-74913ee093a8>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-446-74913ee093a8>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-446-74913ee093a8>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [447]:
#Average Mean Graphs Per Species post - Post

all_sharks_drops_post = drops_post[["All_GR","All_BT","All_SS"]]
all_sharks_drops_post["All_GR"] = all_sharks_drops_post["All_GR"] / GR_sharks_feeding_post
all_sharks_drops_post["All_BT"] = all_sharks_drops_post["All_BT"] / BT_sharks_feeding_post
all_sharks_drops_post["All_SS"] = all_sharks_drops_post["All_SS"] / SS_sharks_feeding_post
all_sharks_drops_post.columns = ["Grey Reef","Black Tip","Sand Shark"]
drops_post_means = pd.DataFrame(round(all_sharks_drops_post.mean(), 3)).reset_index()
drops_post_means.columns = ["Species","Mean"]
drops_post_means = drops_post_means.sort_values(by = "Mean", ascending = False)
print(drops_post_means)
px.bar(drops_post_means, x = "Species", y = "Mean", title = "Drop Means for Each species on average after reopening")

<ipython-input-447-0b27ca05af53>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-447-0b27ca05af53>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-447-0b27ca05af53>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



      Species   Mean
1   Black Tip  0.597
0   Grey Reef  0.197
2  Sand Shark  0.194


In [448]:
All_Sharks_drops_pre = drops_pre.iloc[:,1:]
All_Sharks_drops_pre["All_GR"] = All_Sharks_drops_pre["All_GR"] / GR_sharks_feeding_pre
All_Sharks_drops_pre["All_BT"] = All_Sharks_drops_pre["All_BT"] / BT_sharks_feeding_pre
All_Sharks_drops_pre["All_SS"] = All_Sharks_drops_pre["All_SS"] / SS_sharks_feeding_pre
All_Sharks_drops_pre["female"] = All_Sharks_drops_pre["female"] / female_sharks_feeding_pre
All_Sharks_drops_pre["male"] = All_Sharks_drops_pre["male"] / male_sharks_feeding_pre
All_Sharks_drops_pre["Total: "] = All_Sharks_drops_pre["Total: "] / total_sharks_feeding_pre
drops_pre_means = pd.DataFrame(round(All_Sharks_drops_pre.mean(), 3)).reset_index()
drops_pre_means.columns = ["Subset","Mean"]
drops_pre_means = drops_pre_means.sort_values(by = "Mean", ascending = False)
ci = {}
for column in drops_pre.iloc[:,1:].columns:
    conf = st.t.interval(alpha=0.95, df=len(drops_pre[column])-1, loc=np.mean(drops_pre[column]), scale=st.sem(drops_pre[column],nan_policy='omit'))
    error = (conf[1]-conf[0])/2
    ci.update({column:error})
ci = pd.DataFrame(list(ci.items()),columns = ['Subset','SE'])
drops_pre_means = pd.merge(drops_pre_means, ci, how = 'outer', on = "Subset")
#drops_pre_means = pd.merge(drops_pre_means,pd.melt(drops_pre.describe().loc[["std"]], var_name = "Subset",value_name = "Standard Deviation"), how = "outer", on = "Subset")
prop_average = drops_pre_means[drops_pre_means["Subset"]=="Total: "]["Mean"].values[0]
Average = []
for i in drops_pre_means.Mean:
    if i > prop_average:
        Average.append("Above Average")
    elif i == prop_average:
        Average.append("Average")
    else:
        Average.append("Below Average")
drops_pre_means["Average"] = Average
max_y = (drops_pre_means["Mean"] + drops_pre_means["SE"]).max() 
#max_y = (drops_pre_means["Mean"] + drops_pre_means["Standard Deviation"]).max() 
px.bar(drops_pre_means, x = "Subset", y = "Mean", color = "Average",text = "Mean", error_y = "SE",title = "Drop Means for each subset with Confidence Intervals before covid").update_traces(textposition='outside')




In [449]:
all_sharks_drops_covid = drops_covid.iloc[:,1:]
all_sharks_drops_covid["All_GR"] = all_sharks_drops_covid["All_GR"] / GR_sharks_feeding_covid
all_sharks_drops_covid["All_BT"] = all_sharks_drops_covid["All_BT"] / BT_sharks_feeding_covid
all_sharks_drops_covid["All_SS"] = all_sharks_drops_covid["All_SS"] / SS_sharks_feeding_covid
all_sharks_drops_covid["female"] = all_sharks_drops_covid["female"] / female_sharks_feeding_covid
all_sharks_drops_covid["male"] = all_sharks_drops_covid["male"] / male_sharks_feeding_covid
all_sharks_drops_covid["Total: "] = all_sharks_drops_covid["Total: "] / total_sharks_feeding_covid
drops_covid_means = pd.DataFrame(round(all_sharks_drops_covid.mean(), 3)).reset_index()
drops_covid_means.columns = ["Subset","Mean"]
drops_covid_means = drops_covid_means.sort_values(by = "Mean", ascending = False)
ci = {}
for column in drops_covid.iloc[:,1:].columns:
    conf = st.t.interval(alpha=0.95, df=len(drops_covid[column])-1, loc=np.mean(drops_covid[column]), scale=st.sem(drops_covid[column],nan_policy='omit'))
    error = (conf[1]-conf[0])/2
    ci.update({column:error})
ci = pd.DataFrame(list(ci.items()),columns = ['Subset','SE'])
drops_covid_means = pd.merge(drops_covid_means, ci, how = 'outer', on = "Subset")
#drops_covid_means = pd.merge(drops_covid_means,pd.melt(drops_covid.describe().loc[["std"]], var_name = "Subset",value_name = "Standard Deviation"), how = "outer", on = "Subset")
prop_average = drops_covid_means[drops_covid_means["Subset"]=="Total: "]["Mean"].values[0]
Average = []
for i in drops_covid_means.Mean:
    if i > prop_average:
        Average.append("Above Average")
    elif i == prop_average:
        Average.append("Average")
    else:
        Average.append("Below Average")
drops_covid_means["Average"] = Average
max_y = (drops_covid_means["Mean"] + drops_covid_means["SE"]).max() 
#max_y = (drops_covid_means["Mean"] + drops_covid_means["Standard Deviation"]).max() 
px.bar(drops_covid_means, x = "Subset", y = "Mean", color = "Average",text = "Mean", error_y = "SE",title = "Drop Means for each subset with Confidence Intervals during shutdown").update_traces(textposition='outside')




In [450]:
All_Sharks_drops_post = drops_post.iloc[:,1:]
All_Sharks_drops_post["All_GR"] = All_Sharks_drops_post["All_GR"] / GR_sharks_feeding_post
All_Sharks_drops_post["All_BT"] = All_Sharks_drops_post["All_BT"] / BT_sharks_feeding_post
All_Sharks_drops_post["All_SS"] = All_Sharks_drops_post["All_SS"] / SS_sharks_feeding_post
All_Sharks_drops_post["female"] = All_Sharks_drops_post["female"] / female_sharks_feeding_post
All_Sharks_drops_post["male"] = All_Sharks_drops_post["male"] / male_sharks_feeding_post
All_Sharks_drops_post["Total: "] = All_Sharks_drops_post["Total: "] / total_sharks_feeding_post
drops_post_means = pd.DataFrame(round(All_Sharks_drops_post.mean(), 3)).reset_index()
drops_post_means.columns = ["Subset","Mean"]
drops_post_means = drops_post_means.sort_values(by = "Mean", ascending = False)
ci = {}
for column in drops_post.iloc[:,1:].columns:
    conf = st.t.interval(alpha=0.95, df=len(drops_post[column])-1, loc=np.mean(drops_post[column]), scale=st.sem(drops_post[column],nan_policy='omit'))
    error = (conf[1]-conf[0])/2
    ci.update({column:error})
ci = pd.DataFrame(list(ci.items()),columns = ['Subset','SE'])
drops_post_means = pd.merge(drops_post_means, ci, how = 'outer', on = "Subset")
#drops_post_means = pd.merge(drops_post_means,pd.melt(drops_post.describe().loc[["std"]], var_name = "Subset",value_name = "Standard Deviation"), how = "outer", on = "Subset")
prop_average = drops_post_means[drops_post_means["Subset"]=="Total: "]["Mean"].values[0]
Average = []
for i in drops_post_means.Mean:
    if i > prop_average:
        Average.append("Above Average")
    elif i == prop_average:
        Average.append("Average")
    else:
        Average.append("Below Average")
drops_post_means["Average"] = Average
max_y = (drops_post_means["Mean"] + drops_post_means["SE"]).max() 
#max_y = (drops_post_means["Mean"] + drops_post_means["Standard Deviation"]).max() 
px.bar(drops_post_means, x = "Subset", y = "Mean", color = "Average",text = "Mean", error_y = "SE",title = "Drop Means for each subset with Confidence Intervals before covid").update_traces(textposition='outside')

In [451]:
drops_pre_prop = drops_pre.iloc[:,1:]
drops_covid_prop = drops_covid.iloc[:,1:]
drops_post_prop = drops_post.iloc[:,1:]


props_pre = {}
for i in drops_pre_prop.columns:
    data = drops_pre_prop[i]
    drops_pre_days = data > 0
    props_pre.update({i:(round(len(data[drops_pre_days])/len(data[data.isna()==False]),2))})
props_pre = pd.DataFrame(list(props_pre.items()),columns = ['Subset of Data','Proportion of Days dropped before Covid'])


props_covid = {}
for i in drops_pre_prop.columns:
    data = drops_pre_prop[i]
    drops_pre_days = data > 0
    props_covid.update({i:(round(len(data[drops_pre_days])/len(data[data.isna()==False]),2))})
props_covid = pd.DataFrame(list(props_covid.items()),columns = ['Subset of Data','Proportion of Days dropped during Shutdown'])


props_post = {}
for i in drops_pre_prop.columns:
    data = drops_pre_prop[i]
    drops_pre_days = data > 0
    props_post.update({i:(round(len(data[drops_pre_days])/len(data[data.isna()==False]),2))})
props_post = pd.DataFrame(list(props_post.items()),columns = ['Subset of Data','Proportion of Days dropped after Reopening'])


print(props_pre.iloc[12].values[1],"is the overall proportion of days where a shark ate before covid")
print(props_covid.iloc[12].values[1],"is the overall proportion of days where a shark ate during shutdown")
print(props_post.iloc[12].values[1],"is the overall proportion of days where a shark ate after aquarium reopened")




0.83 is the overall proportion of days where a shark ate before covid
0.83 is the overall proportion of days where a shark ate during shutdown
0.83 is the overall proportion of days where a shark ate after aquarium reopened


In [452]:
# Proportion before covid
drops_pre_prop_species = props_pre.iloc[15:,:].rename(columns = {"Subset of Data":"Species"}).sort_values(by = "Proportion of Days dropped before Covid", ascending = False)
drops_pre_prop_gender = props_pre.iloc[13:15,:].rename(columns = {"Subset of Data":"Gender"}).sort_values(by = "Proportion of Days dropped before Covid", ascending = False)
drops_pre_prop_sharks = props_pre.iloc[0:12,:].rename(columns = {"Subset of Data":"Shark"}).sort_values(by = "Proportion of Days dropped before Covid", ascending = False)

#Proportions during covid
drops_covid_prop_species = props_covid.iloc[15:,:].rename(columns = {"Subset of Data":"Species"}).sort_values(by = "Proportion of Days dropped during Shutdown", ascending = False)
drops_covid_prop_gender = props_covid.iloc[13:15,:].rename(columns = {"Subset of Data":"Gender"}).sort_values(by = "Proportion of Days dropped during Shutdown", ascending = False)
drops_covid_prop_sharks = props_covid.iloc[0:12,:].rename(columns = {"Subset of Data":"Shark"}).sort_values(by = "Proportion of Days dropped during Shutdown", ascending = False)

#Proportions after covid
drops_post_prop_species = props_post.iloc[15:,:].rename(columns = {"Subset of Data":"Species"}).sort_values(by = "Proportion of Days dropped after Reopening", ascending = False)
drops_post_prop_gender = props_post.iloc[13:15,:].rename(columns = {"Subset of Data":"Gender"}).sort_values(by = "Proportion of Days dropped after Reopening", ascending = False)
drops_post_prop_sharks = props_post.iloc[0:12,:].rename(columns = {"Subset of Data":"Shark"}).sort_values(by = "Proportion of Days dropped after Reopening", ascending = False)

In [453]:
px.bar(drops_pre_prop_species, x = "Species", y = "Proportion of Days dropped before Covid", title = "What proportion of days does each species Drop before covid?")

In [454]:
px.bar(drops_covid_prop_species, x = "Species", y = "Proportion of Days dropped during Shutdown", title = "What proportion of days does each species Drop during Shutdown?")

In [455]:
px.bar(drops_post_prop_species, x = "Species", y = "Proportion of Days dropped after Reopening", title = "What proportion of days does each species Drop after Reopening?")

In [456]:
px.bar(drops_pre_prop_gender, x = "Gender", y = "Proportion of Days dropped before Covid", title = "What proportion of days does each gender Drop before covid?")

In [457]:
px.bar(drops_covid_prop_gender, x = "Gender", y = "Proportion of Days dropped during Shutdown", title = "What proportion of days does each gender Drop during Shutdown?")

In [458]:
px.bar(drops_post_prop_gender, x = "Gender", y = "Proportion of Days dropped after Reopening", title = "What proportion of days does each gender Drop after Reopening?")

In [459]:
px.bar(drops_pre_prop_sharks, x = "Shark", y = "Proportion of Days dropped before Covid", title = "What proportion of days does each shark Drop before covid?")

In [460]:
px.bar(drops_covid_prop_sharks, x = "Shark", y = "Proportion of Days dropped during Shutdown", title = "What proportion of days does each shark Drop during the Shutdown?")

In [461]:
px.bar(drops_post_prop_sharks, x = "Shark", y = "Proportion of Days dropped after Reopening", title = "What proportion of days does each shark Drop after Reopening?")